In [62]:
import sys
import copy
import time

import pandas as pd
import numpy as np
import scipy as sp

import PyFBA

In [3]:
annotFile = "/Users/feng626/workspace/test/wheeler/str001-annotated/str001.tsv"
annot = pd.read_csv(annotFile, sep='\t')
annot

In [21]:
allReactions = pd.read_csv("/Users/feng626/Downloads/reactions.tsv", sep='\t')
allReactions

,id,abbreviation,name,code,stoichiometry,is_transport,equation,definition,reversibility,direction,...,aliases,ec_numbers,deltag,deltagerr,compound_ids,status,is_obsolete,linked_reaction,notes,source
0,rxn00001,R00004,diphosphate phosphohydrolase,(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd0...,"-1:cpd00001:0:0:""H2O"";-1:cpd00012:0:0:""PPi"";2:...",0,(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd0...,(1) H2O[0] + (1) PPi[0] <=> (2) Phosphate[0] +...,>,=,...,AraCyc: INORGPYROPHOSPHAT-RXN|BiGG: IPP1; PPA;...,3.6.1.1,-3.46,0.05,cpd00001;cpd00009;cpd00012;cpd00067,OK,0,rxn27946;rxn27947;rxn27948;rxn32487;rxn38157;r...,GCC|HB|EQC|EQU,Primary Database
1,rxn00002,R00005,urea-1-carboxylate amidohydrolase,(1) cpd00001[0] + (1) cpd00742[0] <=> (2) cpd0...,"-1:cpd00001:0:0:""H2O"";-3:cpd00067:0:0:""H+"";-1:...",0,(1) cpd00001[0] + (3) cpd00067[0] + (1) cpd007...,(1) H2O[0] + (3) H+[0] + (1) Allophanate[0] =>...,>,>,...,AraCyc: ALLOPHANATE-HYDROLASE-RXN|BiGG: ALPHNH...,3.5.1.54,-20.14,1.86,cpd00001;cpd00011;cpd00013;cpd00067;cpd00742,OK,0,rxn30346;rxn35525,GCC|EQC|EQU,Primary Database
2,rxn00003,R00006,pyruvate:pyruvate acetaldehydetransferase (dec...,(1) cpd00011[0] + (1) cpd00668[0] <=> (2) cpd0...,"-1:cpd00011:0:0:""CO2"";-1:cpd00668:0:0:""ALCTT"";...",0,(1) cpd00011[0] + (1) cpd00668[0] <= (2) cpd00...,(1) CO2[0] + (1) ALCTT[0] <= (2) Pyruvate[0] +...,<,<,...,AlgaGEM: R_R00006_c|AraGEM: R_R00006_c|BiGG: I...,2.2.1.6,8.27,0.90,cpd00011;cpd00020;cpd00067;cpd00668,OK,0,rxn30144;rxn33164,GCC|EQC|EQU,Primary Database
3,rxn00004,R00008,4-hydroxy-4-methyl-2-oxoglutarate pyruvate-lya...,(1) cpd02570[0] <=> (2) cpd00020[0],"-1:cpd02570:0:0:""Parapyruvate"";2:cpd00020:0:0:...",0,(1) cpd02570[0] <=> (2) cpd00020[0],(1) Parapyruvate[0] <=> (2) Pyruvate[0],=,=,...,KEGG: R00008|Name: 4-Hydroxy-4-methyl-2-oxoglu...,4.1.3.17,4.49,0.57,cpd00020;cpd02570,OK,0,NaN,GCC|EQC|EQU,Primary Database
4,rxn00006,R00009,hydrogen-peroxide:hydrogen-peroxide oxidoreduc...,(2) cpd00025[0] <=> (2) cpd00001[0] + (1) cpd0...,"-2:cpd00025:0:0:""H2O2"";2:cpd00001:0:0:""H2O"";1:...",0,(2) cpd00025[0] => (2) cpd00001[0] + (1) cpd00...,(2) H2O2[0] => (2) H2O[0] + (1) O2[0],>,>,...,AraCyc: CATAL-RXN|BiGG: CAT; CATp; CTA1; CTT1|...,1.11.1.21|1.11.1.6,-46.06,1.64,cpd00001;cpd00007;cpd00025,OK,0,rxn19264;rxn22404;rxn27744;rxn31381,GCC|EQC|EQU,Primary Database
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43769,rxn48571,R02164,Succinate:ubiquinone oxidoreductase,(1) cpd00036[0] + (1) cpd11669[0] <=> (1) cpd0...,"-1:cpd00036:0:0:""Succinate"";-1:cpd11669:0:0:""Q...",0,(1) cpd00036[0] + (1) cpd11669[0] <=> (1) cpd0...,(1) Succinate[0] + (1) Q[0] <=> (1) Fumarate[0...,?,=,...,DF_Athaliana: Complex_II|JM_Creinhardtii: JM_C...,1.3.5.1,10000000.00,10000000.00,cpd00036;cpd00106;cpd11669;cpd15291,MI:C:20/H:32,0,rxn30643,EQC|EQU|GCP,Primary Database
43770,rxn48572,R04993,S-adenosyl-L-methione:demethylmenaquinone meth...,(1) cpd00017[0] + (1) cpd11425[0] <=> (1) cpd0...,"-1:cpd00017:0:0:""S-Adenosyl-L-methionine"";-1:c...",0,(1) cpd00017[0] + (1) cpd00067[0] + (1) cpd114...,(1) S-Adenosyl-L-methionine[0] + (1) H+[0] + (...,?,>,...,AlgaGEM: R_R04993_c|AraGEM: R_R04993_c|Maize_C...,2.1.1.163,10000000.00,10000000.00,cpd00017;cpd00019;cpd00067;cpd11425;cpd15500,MI:C:30/H:46|CI:-2,0,NaN,EQC|GCP,Primary Database
43771,rxn48573,R05635,Formyl-L-methionyl peptide amidohydrolase,(1) cpd00001[0] + (1) cpd37291[0] <=> (1) cpd0...,"-1:cpd00001:0:0:""H2O"";-1:cpd37291:0:0:FORMYL-L...",0,(1) cpd00001[0] + (1) cpd37291[0] <=> (1) cpd0...,(1) H2O[0] + (1) FORMYL-L-METHIONYL-PEPTIDE[0]...,?,=,...,AraCyc: 3.5.1.88-RXN|BrachyCyc: 3.5.1.88-RXN|C...,3.5.1.88|EC-3.5.1.88,10000000.00,10000000.00,cpd00001;cpd00047;cpd37290;cpd37291,OK,0,rxn27562;rxn27563,HB|GCP|EQP,Primary Database
43772,rxn48574,R03874,Peptidylamidoglycolate peptidylamide-lyase,(1) cpd37292[0] <=> (1) cpd00040[0] + (1) cpd1...,-1:cpd37292:0:0:Peptidyl(2-hydroxyglycine);1:c...,0,(1) cpd37292[0] => (1) cpd0

In [31]:
reactionAnnot = allReactions[["id", "name", "aliases", "ec_numbers"]]
reactionAnnot

,id,name,aliases,ec_numbers
0,rxn00001,diphosphate phosphohydrolase,AraCyc: INORGPYROPHOSPHAT-RXN|BiGG: IPP1; PPA;...,3.6.1.1
1,rxn00002,urea-1-carboxylate amidohydrolase,AraCyc: ALLOPHANATE-HYDROLASE-RXN|BiGG: ALPHNH...,3.5.1.54
2,rxn00003,pyruvate:pyruvate acetaldehydetransferase (dec...,AlgaGEM: R_R00006_c|AraGEM: R_R00006_c|BiGG: I...,2.2.1.6
3,rxn00004,4-hydroxy-4-methyl-2-oxoglutarate pyruvate-lya...,KEGG: R00008|Name: 4-Hydroxy-4-methyl-2-oxoglu...,4.1.3.17
4,rxn00006,hydrogen-peroxide:hydrogen-peroxide oxidoreduc...,AraCyc: CATAL-RXN|BiGG: CAT; CATp; CTA1; CTT1|...,1.11.1.21|1.11.1.6
...,...,...,...,...
43769,rxn48571,Succinate:ubiquinone oxidoreductase,DF_Athaliana: Complex_II|JM_Creinhardtii: JM_C...,1.3.5.1
43770,rxn48572,S-adenosyl-L-methione:demethylmenaquinone meth...,AlgaGEM: R_R04993_c|AraGEM: R_R04993_c|Maize_C...,2.1.1.163
43771,rxn48573,Formyl-L-methionyl peptide amidohydrolase,AraCyc: 3.5.1.88-RXN|BrachyCyc: 3.5.1.88-RXN|C...,3.5.1.88|EC-3.5.1.88
43772,rxn48574,Peptidylamidoglycolate peptidylamide-lyase,KEGG: R03874,NaN


In [64]:
t0 = time.time()
reactions = []
functions = []
for product in annot["product"]:
    if product != "hypothetical protein":
        functions.append(product)
        for i in range(allReactions.shape[0]):
            if not (pd.isna(allReactions["name"][i]) or pd.isna(allReactions["aliases"][i])):
                if product in allReactions["name"][i] or product in allReactions["aliases"][i] :
                    reactions.append(allReactions["id"][i])
t1 = time.time()
print(t1 - t0)

3268.517800807953


In [65]:
print(len(functions))
print(len(set(reactions)))
len(reactions)

3268
1263


2073

In [67]:
rastFile = "/Users/feng626/workspace/test/wheeler/str001-RAST_annotation.txt"
rast = pd.read_csv(rastFile, sep='\t')
rast

,contig_id,feature_id,type,location,start,stop,strand,function,aliases,figfam,evidence_codes,nucleotide_sequence,aa_sequence
0,str001,fig|6666666.722188.peg.1,peg,str001_1215_241,1215,241,-,"Transcriptional regulator, LysR family",NaN,NaN,NaN,atgtggatccctgtcattagtggaacaaatagtctctgctcgcacg...,MWIPVISGTNSLCSHAPWEAHPVNLARLDLNLVVALRALLEERNVT...
1,str001,fig|6666666.722188.peg.2,peg,str001_1282_2274,1282,2274,+,Putative hydrolase,NaN,NaN,NaN,gtgaaatctgaatccgcttctgatcttttctctggccctttcgctc...,MKSESASDLFSGPFALGTFSAADGPAFPGLVDPEGQVLDLRTALAD...
2,str001,fig|6666666.722188.peg.3,peg,str001_2271_3227,2271,3227,+,putative cyclase SCIF3.09c,NaN,NaN,NaN,gtgaaccgtaccgaccccgagggcgcgatcgcccaggccgccaagg...,MNRTDPEGAIAQAAKECSNWGRWGQDDRLGTVNFLDEAKRREGAAL...
3,str001,fig|6666666.722188.peg.4,peg,str001_3268_4398,3268,4398,+,Putative oxidoreductase,NaN,NaN,NaN,atgagcacaccccgcaaggttctggtcatcggcggcggggcggccg...,MSTPRKVLVIGGGAAGNAATILLRRAGIDVDLVEAKEDWNATAGSG...
4,str001,fig|6666666.722188.peg.5,peg,str001_4395_5399,4395,5399,+,2-amino-3-carboxymuconate-6-semialdehyde decar...,NaN,NaN,NaN,gtgaccgcgccgagcgtggacgtccacgcccacgtcctgctgccgc...,MTAPSVDVHAHVLLPQIEALVAGLPGHAEAKALDARRNGPDALAVS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7569,str001scaf0433,fig|6666666.722188.peg.7499,peg,str001scaf0433_1221_2372,1221,2372,+,ADP-ribosylglycohydrolase,NaN,NaN,NaN,atgacacccacactggatgaccggatcaccggcagtctcgtcgggg...,MTPTLDDRITGSLVGAAVGDALGGPVEGYTPEQILERHGGRVTGVV...
7570,str001scaf0433,fig|6666666.722188.peg.7500,peg,str001scaf0433_2369_2530,2369,2530,+,putative membrane protein,NaN,NaN,NaN,gtgaacctccgcctgacctgggtccaaccggaggacctggtgggcc...,MNLRLTWVQPEDLVGHELRQAREDGRDASALAARWAAAGGPPAPES...
7571,str001scaf0434,fig|6666666.722188.peg.7501,peg,str001scaf0434_118_2,118,2,-,hypothetical protein,NaN,NaN,NaN,atgccgacctgcccgaacggacaccagtcggtctccgacgactggt...,MPTCPNGHQSVSDDWCPFGQVGMSVLPPLVVRPGGRALP
7572,str001scaf0434,fig|6666666.722188.peg.7502,peg,str001scaf0434_1491_160,1491,160,-,conserved hypothetical protein SC6D10.11,NaN,NaN,NaN,atggcgaacttctccaagccgaccgcgccctcgttctcggtggacg...,MANFSKPTAPSFSVDVYQNEFLPEGGRDVSAIVTVTSTGGGTAGQR...


In [68]:
t0Rast = time.time()
reactionsRast = []
functionsRast = []
for product in rast["function"]:
    if product != "hypothetical protein":
        functionsRast.append(product)
        for i in range(allReactions.shape[0]):
            if not (pd.isna(allReactions["name"][i]) or pd.isna(allReactions["aliases"][i])):
                if product in allReactions["name"][i] or product in allReactions["aliases"][i] :
                    reactionsRast.append(allReactions["id"][i])
t1Rast = time.time()
print(t1Rast - t0Rast)

4239.447870969772


In [69]:
print(len(functionsRast))
print(len(set(reactionsRast)))
len(reactions)

4799
1261


2073

In [70]:
roles = set(annot["function"])

In [ ]:
testRxns = PyFBA.filters.roles_to_reactions(roles)